#### 1. Configurando o ambiente

In [10]:
# Instalando os pacotes necessários
# pip install -U ipykernel    # Kernel do Jupyter
# pip install -U jupyter      # Jupyter
# pip install openai          # API do OpenAI

In [30]:
# Configuração básica
import os                          # Biblioteca para manipulação de arquivos
from openai import OpenAI          # Biblioteca para uso da API do OpenAI       
import json                        # Biblioteca para manipulação de arquivos JSON

# Carregando variáveis de ambiente
with open('azure_openai_keys.json', 'r') as f:                  # Carregando as chaves de API   
    keys = json.load(f)                                         

# Configuração da API 

api_base = keys["AZURE_OPENAI_ENDPOINT"]                 # Endpoint da API  (azure, gcp, aws)
api_key = keys["AZURE_OPENAI_API_KEY"]                   # Chave de API do OpenAI

#### 2. Chamada da API

In [32]:
# Exemplo de chamada à API para geração de texto

# Inicialização do cliente OpenAI com as configurações

client = OpenAI(
    api_key=api_key,
    azure_endpoint=api_base,
    api_version="2020-05-03"    
)

# Chamada à API para geração de texto  
try:
    response = client.completions.create(
        model="gpt-4o",                         # Nome do modelo
        temperature=0.7,                        # Controla a criatividade do texto gerado  (0.0 - 1.0)
        max_tokens=100,                         # Número máximo de tokens a serem gerados no texto
        prompt="Explique o que é Inteligência Artificial Generativa."      # Texto de entrada para geração de texto
    )

    print(response.choices[0].text.strip())     # Exibindo o texto gerado pela API

except Exception as e:
    print(f"Erro ao chamar a API: {e}")         # Exibindo o erro caso ocorra   

TypeError: OpenAI.__init__() got an unexpected keyword argument 'azure_endpoint'

#### 3. Exibir modelos disponiveis 

In [34]:
import openai
openai.api_key = keys["AZURE_OPENAI_API_KEY"]

# Listando modelos disponíveis

models = openai.Model.list()    # Listando modelos disponíveis
for model in models.data:
    print(model.id)

APIRemovedInV1: 

You tried to access openai.Model, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


#### 4. Personalizar respostas com parâmetros

In [ ]:
# Ajustando parâmetros para personalizar a resposta

response = openai.Completion.create(        # Chamada à API para geração de texto   
    engine="davinci",                       # Nome do modelo       
    prompt="Escreva uma história curta sobre uma Capivara.",    # Texto de entrada
    temperature=0.5,                        # Controla a criatividade do texto gerado  (0.0 - 1.0)
    max_tokens=150,                         # Número máximo de tokens a serem gerados no texto
    top_p=1.0,                              # Controla a probabilidade de escolha das palavras geradas (0.0 - 1.0)  
    frequency_penalty=0.5,                  # Controla a frequência das palavras geradas (0.0 - 1.0)
    presence_penalty=0.5                    # Controla a presença das palavras geradas (0.0 - 1.0)
)

print(response.choices[0].text.strip())     # Exibindo o texto gerado

#### 5. Trabalhando com conversas (chatbots)

In [ ]:
# Exemplo de conversa com o modelo
messages = [    #   Mensagens trocadas entre o usuário e o modelo
    {"role": "system", "content": "Você é um especialista em pescarias."},
    {"role": "user", "content": "De onde é o peixe Tambaqui?"}
]

response = openai.ChatCompletion.create( #  Chamada à API para conversação  com o modelo
    engine="gpt-4",             # Nome do modelo
    messages=messages,          # Mensagens trocadas entre o usuário e o modelo
    temperature=0.7,            # Controla a criatividade do texto gerado  (0.0 - 1.0)
    max_tokens=100              # Número máximo de tokens a serem gerados no texto    
)

print(response.choices[0].message['content'])   # Exibindo a resposta do modelo ao usuário

#### 6. Integração com aplicações

In [ ]:
# Integração simples em uma aplicação Flask
from flask import Flask, request, jsonify
import openai 

app = Flask(__name__)       # Inicializando a aplicação Flask

@app.route('/generate', methods=['POST'])       # Rota para geração de texto

def generate_text():                            # Função para geração de texto 
    data = request.json                         # Recebendo dados da requisição 
    prompt = data.get('prompt')                 # Obtendo o prompt da requisição

    response = openai.Completion.create(        # Chamada à API para geração de texto  com base no prompt
        engine="davinci",                       # Nome do modelo
        prompt=prompt,                          # Prompt para geração de texto
        temperature=0.7,                        # Controla a criatividade do texto gerado  (0.0 - 1.0) 
        max_tokens=100                          # Número máximo de tokens a serem gerados no texto
    )

    return jsonify({"response": response.choices[0].text.strip()})        # Retornando a resposta  gerada  pela API

if __name__ == '__main__':          # Inicializando a aplicação Flask
    app.run(debug=True)             # Executando a aplicação Flask